In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import matplotlib.cm as cm
from scipy.interpolate import interp1d
from salishsea_tools import (nc_tools, gsw_calls,viz_tools)

/home/ssahu/anaconda3/lib/python3.5/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
west_bdy_3d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_west_twoless_m04.nc');

In [3]:
nc_tools.show_variables(west_bdy_3d)

odict_keys(['xb', 'yb', 'deptht', 'time_counter', 'vozocrtx', 'vomecrty', 'votemper', 'vosaline'])


In [4]:
vozocrtx_full = west_bdy_3d.variables['vozocrtx'][...];
vomecrty_full = west_bdy_3d.variables['vomecrty'][...];
votemper_full = west_bdy_3d.variables['votemper'][...];
vosaline_full = west_bdy_3d.variables['vosaline'][...];
deptht = west_bdy_3d.variables['deptht'][...];



print(vozocrtx_full.shape, vomecrty_full.shape, deptht.shape)

Nt = vozocrtx_full.shape[0]
Nz = vozocrtx_full.shape[1]
Ny = vozocrtx_full.shape[2]
Nx = vozocrtx_full.shape[3]

vozocrtx_new = np.reshape(a=vozocrtx_full, newshape= (Nt,Nz,1,Nx*Ny))
vomecrty_new = np.reshape(a=vomecrty_full, newshape= (Nt,Nz,1,Nx*Ny))
votemper_new = np.reshape(a=votemper_full, newshape= (Nt,Nz,1,Nx*Ny))
vosaline_new = np.reshape(a=vosaline_full, newshape= (Nt,Nz,1,Nx*Ny))

print(votemper_new.shape)

(43, 27, 4, 98) (43, 27, 4, 98) (27,)
(43, 27, 1, 392)


In [7]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_west_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', vozocrtx_new.shape[3]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('deptht', vozocrtx_new.shape[1]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along left boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'one y index across all of left boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[...] = vozocrtx_new[...];
vomecrty[...] = vomecrty_new[...];
votemper[...] = votemper_new[...];
vosaline[...] = vosaline_new[...];


deptht[:] = deptht[:];


bdy_file.close()

print(vozocrtx_new.shape)

(43, 27, 1, 392)


### The north and the east boundary strips have to be flipped in order for them to be arranged in the increasing order of nbr (as required by the NEMO input files)

In [10]:
north_bdy_3d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_north_twoless_m04.nc');

In [15]:
vozocrtx_full = north_bdy_3d.variables['vozocrtx'][...];
vomecrty_full = north_bdy_3d.variables['vomecrty'][...];
votemper_full = north_bdy_3d.variables['votemper'][...];
vosaline_full = north_bdy_3d.variables['vosaline'][...]
deptht = north_bdy_3d.variables['deptht'][...];


print(vozocrtx_full.shape, vomecrty_full.shape, deptht.shape)

Nt = vozocrtx_full.shape[0]
Nz = vozocrtx_full.shape[1]
Ny = vozocrtx_full.shape[2]
Nx = vozocrtx_full.shape[3]


vozocrtx_flipped = np.flip(vozocrtx_full, axis=2)
vomecrty_flipped = np.flip(vomecrty_full, axis=2)
votemper_flipped = np.flip(votemper_full, axis=2)
vosaline_flipped = np.flip(vosaline_full, axis=2)



vozocrtx_new = np.reshape(a=vozocrtx_flipped, newshape= (Nt,Nz,1,Nx*Ny))
vomecrty_new = np.reshape(a=vomecrty_flipped, newshape= (Nt,Nz,1,Nx*Ny))
votemper_new = np.reshape(a=votemper_flipped, newshape= (Nt,Nz,1,Nx*Ny))
vosaline_new = np.reshape(a=vosaline_flipped, newshape= (Nt,Nz,1,Nx*Ny))

print(votemper_new.shape)

(43, 27, 4, 68) (43, 27, 4, 68) (27,)
(43, 27, 1, 272)


In [16]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_north_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', vozocrtx_new.shape[3]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('deptht', vozocrtx_new.shape[1]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along north boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'one y index across all of north boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[...] = vozocrtx_new[...];
vomecrty[...] = vomecrty_new[...];
votemper[...] = votemper_new[...];
vosaline[...] = vosaline_new[...];

print(vozocrtx.shape)

bdy_file.close()

(43, 27, 1, 272)


In [17]:
east_bdy_3d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_right_twoless_m04.nc');

In [18]:
vozocrtx_full = east_bdy_3d.variables['vozocrtx'][...];
vomecrty_full = east_bdy_3d.variables['vomecrty'][...];
votemper_full = east_bdy_3d.variables['votemper'][...];
vosaline_full = east_bdy_3d.variables['vosaline'][...];
deptht = east_bdy_3d.variables['deptht'][...];


print(vozocrtx_full.shape, vomecrty_full.shape, deptht.shape)


Nt = vozocrtx_full.shape[0]
Nz = vozocrtx_full.shape[1]
Ny = vozocrtx_full.shape[2]
Nx = vozocrtx_full.shape[3]


vozocrtx_flipped = np.flip(vozocrtx_full, axis=2)
vomecrty_flipped = np.flip(vomecrty_full, axis=2)
votemper_flipped = np.flip(votemper_full, axis=2)
vosaline_flipped = np.flip(vosaline_full, axis=2)



vozocrtx_new = np.reshape(a=vozocrtx_flipped, newshape= (Nt,Nz,1,Nx*Ny))
vomecrty_new = np.reshape(a=vomecrty_flipped, newshape= (Nt,Nz,1,Nx*Ny))
votemper_new = np.reshape(a=votemper_flipped, newshape= (Nt,Nz,1,Nx*Ny))
vosaline_new = np.reshape(a=vosaline_flipped, newshape= (Nt,Nz,1,Nx*Ny))

print(votemper_new.shape)

(43, 27, 4, 98) (43, 27, 4, 98) (27,)
(43, 27, 1, 392)


In [19]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_east_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', vozocrtx_new.shape[3]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('deptht', vozocrtx_new.shape[1]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along east boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'one y index across all of east boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[...] = vozocrtx_new[...];
vomecrty[...] = vomecrty_new[...];
votemper[...] = votemper_new[...];
vosaline[...] = vosaline_new[...];

print(vozocrtx.shape)

bdy_file.close()

(43, 27, 1, 392)


In [20]:
south_bdy_3d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_south_twoless_m04.nc');

In [21]:
vozocrtx_full = south_bdy_3d.variables['vozocrtx'][...];
vomecrty_full = south_bdy_3d.variables['vomecrty'][...];
votemper_full = south_bdy_3d.variables['votemper'][...];
vosaline_full = south_bdy_3d.variables['vosaline'][...];
deptht = south_bdy_3d.variables['deptht'][...];


print(vozocrtx_full.shape, vomecrty_full.shape, deptht.shape)

Nt = vozocrtx_full.shape[0]
Nz = vozocrtx_full.shape[1]
Ny = vozocrtx_full.shape[2]
Nx = vozocrtx_full.shape[3]

vozocrtx_new = np.reshape(a=vozocrtx_full, newshape= (Nt,Nz,1,Nx*Ny))
vomecrty_new = np.reshape(a=vomecrty_full, newshape= (Nt,Nz,1,Nx*Ny))
votemper_new = np.reshape(a=votemper_full, newshape= (Nt,Nz,1,Nx*Ny))
vosaline_new = np.reshape(a=vosaline_full, newshape= (Nt,Nz,1,Nx*Ny))

print(votemper_new.shape)

(43, 27, 4, 68) (43, 27, 4, 68) (27,)
(43, 27, 1, 272)


In [24]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_south_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', vozocrtx_new.shape[3]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('deptht', vozocrtx_new.shape[1]);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'x indices along south boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'one y index across all of south boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';


vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xbT'), zlib=True);

vozocrtx[...] = vozocrtx_new[...];
vomecrty[...] = vomecrty_new[...];
votemper[...] = votemper_new[...];
vosaline[...] = vosaline_new[...];

print(vozocrtx.shape)

bdy_file.close()

(43, 27, 1, 272)
